# Week 7 — Part 03: Error Handling Lab

**Estimated time:** 60–90 minutes

---

## Pre-study (Self-learn)

Foundamental Course assumes Self-learn is complete. If you need a refresher on exceptions and debugging:

- [Foundamental Course Pre-study index](../PRESTUDY.md)
- [Self-learn — Modules and exception handling](../self_learn/Chapters/2/02_modules_exceptions.md)

---

## What success looks like (end of Part 03)

- You can write error messages that tell users what to do.
- You can distinguish user errors from system errors.
- You can log detailed errors while showing friendly messages.

### Checkpoint

After running this notebook:
- You can raise helpful error messages
- You can implement error handling strategies

## Learning Objectives

- Write actionable error messages
- Distinguish error types
- Implement proper error handling

## Overview

In this lab you will make failures explainable.

You will:

- validate inputs early (file exists, non-empty, required columns)
- classify exceptions into simple categories
- emit short, actionable user-facing messages

If you want the deeper discussion about errors as “product surface”, use the Self-learn links at the top of the notebook.

In [ ]:
from pathlib import Path
from typing import List


def require_file(path: str) -> Path:
    p = Path(path).expanduser()
    if not p.exists():
        raise FileNotFoundError(
            "Input file not found: %s. Check the path or run --help for usage." % p
        )
    if p.stat().st_size == 0:
        raise ValueError("Input file is empty: %s. Provide a non-empty CSV." % p)
    return p


def require_columns(df, required: List[str]) -> None:
    missing = [c for c in required if c not in df.columns]
    if missing:
        raise ValueError(
            "Missing required columns: %s. Update your schema or fix the input file." % missing
        )


print("error helpers ready")

In [ ]:
def classify_error(exc: Exception) -> str:
    # TODO: categorize errors into input/config/external/output types.
    if isinstance(exc, (FileNotFoundError, ValueError)):
        return "input"
    if isinstance(exc, RuntimeError):
        return "config"
    return "external"


def user_facing_message(exc: Exception) -> str:
    # TODO: map error type to a short, actionable message.
    kind = classify_error(exc)
    if kind == "input":
        return "Input error: %s" % str(exc)
    if kind == "config":
        return "Config error: %s" % str(exc)
    if kind == "output":
        return "Output error: %s" % str(exc)
    return "External error: %s" % str(exc)


print(user_facing_message(FileNotFoundError("missing.csv")))
print(user_facing_message(RuntimeError("Missing API_KEY")))
print("Implement classify_error() and user_facing_message().")

## Self-check

- If a beginner runs your project wrong, does the error message teach them what to do?

## References

- Python errors/exceptions: https://docs.python.org/3/tutorial/errors.html

## Appendix: Solutions (peek only after trying)

Reference implementations for `classify_error` and `user_facing_message`.

In [ ]:
def classify_error(exc: Exception) -> str:
    if isinstance(exc, FileNotFoundError):
        return "input"
    if isinstance(exc, ValueError):
        return "input"
    msg = str(exc).lower()
    if "api_key" in msg or "missing" in msg and "env" in msg:
        return "config"
    if "json" in msg or "schema" in msg or "parse" in msg:
        return "output"
    return "external"


def user_facing_message(exc: Exception) -> str:
    kind = classify_error(exc)
    if kind == "input":
        return "%s\nNext: check the input path/CSV schema." % str(exc)
    if kind == "config":
        return "%s\nNext: set required env vars (e.g. API_KEY) and retry." % str(exc)
    if kind == "output":
        return "%s\nNext: inspect the raw model output saved under output/." % str(exc)
    return "%s\nNext: retry with backoff or check provider status." % str(exc)


print("solution_message:", user_facing_message(ValueError("invalid JSON")))